In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import math
from matplotlib import pyplot
from datetime import date


from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, Concatenate
from sklearn.metrics import f1_score, accuracy_score


Using TensorFlow backend.


In [3]:
df_train = pd.read_csv("C:/00ETH/ml4h/Project1/archive/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("C:/00ETH/ml4h/Project1/archive/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]


In [4]:
# Transformation 1 -> smoothing original time series

def smooth_data(data, window_size):
    rolling_transform = data.rolling(window=window_size)
    rolling_mean = rolling_transform.mean()
    return rolling_mean    

X_smooth_small = smooth_data(df_train, 3)
X_smooth_medium = smooth_data(df_train, 9)
X_smooth_large = smooth_data(df_train, 15)

X_test_smooth_small = smooth_data(df_test, 3)
X_test_smooth_medium = smooth_data(df_test, 9)
X_test_smooth_large = smooth_data(df_test, 15)

X_smooth_small = X_smooth_small.sample(frac=1)
X_smooth_medium = X_smooth_medium.sample(frac=1)
X_smooth_large = X_smooth_large.sample(frac=1)

X_test_smooth_small = X_test_smooth_small.sample(frac=1)
X_test_smooth_medium = X_test_smooth_medium.sample(frac=1)
X_test_smooth_large = X_test_smooth_large.sample(frac=1)

X_smooth_small = np.array(X_smooth_small[list(range(187))].values)[..., np.newaxis]
X_smooth_medium = np.array(X_smooth_medium[list(range(187))].values)[..., np.newaxis]
X_smooth_large = np.array(X_smooth_large[list(range(187))].values)[..., np.newaxis]

X_test_smooth_small = np.array(X_test_smooth_small[list(range(187))].values)[..., np.newaxis]
X_test_smooth_medium = np.array(X_test_smooth_medium[list(range(187))].values)[..., np.newaxis]
X_test_smooth_large = np.array(X_test_smooth_large[list(range(187))].values)[..., np.newaxis]

In [43]:
# Transformation 2 -> downsample original time series
def downsample_data(data, factor, sample_rate):
    start = date(2000, 1, 1)
    end = start + dt.timedelta(days=data.shape[1]-1)
    index = pd.date_range(start, end)
    s = pd.Series(data[0,:,0], index=index)
    s = s.resample(sample_rate)
    sample_mean = s.mean()
    data_return = np.zeros([data.shape[0], sample_mean.shape[0]])
    for i in range(1,data.shape[0]-1):
        s = pd.Series(data[i,:,0], index=index)
        s = s.resample(sample_rate)
        sample_mean = s.mean()
        sample_mean = sample_mean.to_numpy()
        data_return[i,:] = sample_mean
    return data_return

X_sample_small = downsample_data(X, 1.2, '1.2D')
X_sample_medium = downsample_data(X, 1.4, '1.4D')
X_sample_large = downsample_data(X, 1.6, '1.6D')

In [44]:
X_test_sample_small = downsample_data(X_test, 1.2, '1.2D')
X_test_sample_medium = downsample_data(X_test, 1.4, '1.4D')
X_test_sample_large = downsample_data(X_test, 1.6, '1.6D')

In [45]:
X_sample_small = X_sample_small[..., np.newaxis]
X_sample_medium = X_sample_medium[..., np.newaxis]
X_sample_large = X_sample_large[..., np.newaxis]

X_test_sample_small = X_test_sample_small[..., np.newaxis]
X_test_sample_medium = X_test_sample_medium[..., np.newaxis]
X_test_sample_large = X_test_sample_large[..., np.newaxis]
    

In [46]:
print(X.shape)
print(X_smooth_small.shape)
print(X_sample_small.shape)
print(X_sample_medium.shape)
print(X_sample_large.shape)

(87554, 187, 1)
(87554, 187, 1)
(87554, 156, 1)
(87554, 133, 1)
(87554, 117, 1)


In [47]:
def get_base_model(input_len, filter_size):
        inp = Input(shape=(input_len, 1))
        img_1 = Convolution1D(int(filter_size/2), kernel_size=4, activation=activations.relu, padding="valid")(inp)
        img_1 = Convolution1D(int(filter_size/2), kernel_size=4, activation=activations.relu, padding="valid")(img_1)
        img_1 = MaxPool1D(pool_size=2)(img_1)
        img_1 = Dropout(rate=0.1)(img_1)
        img_1 = Convolution1D(filter_size, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
        img_1 = Convolution1D(filter_size, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
        img_1 = MaxPool1D(pool_size=2)(img_1)
        img_1 = Dropout(rate=0.1)(img_1)
        img_1 = Convolution1D(filter_size, kernel_size=2, activation=activations.relu, padding="valid")(img_1)
        img_1 = Convolution1D(filter_size, kernel_size=2, activation=activations.relu, padding="valid")(img_1)
        img_1 = MaxPool1D(pool_size=2)(img_1)
        img_1 = Dropout(rate=0.1)(img_1)
        
        dense = Dense(filter_size*2,activation=activations.relu, name="dense_1")(img_1)
        dense = Dropout(0.3)(dense)
        model = models.Model(inputs = inp, outputs = dense)
        return model

In [56]:
def get_base_model2(input_len, filter_size):
        inp = Input(shape=(input_len, 1))
        img_1 = Convolution1D(int(filter_size/2), kernel_size=4, activation=activations.relu, padding="valid")(inp)
        img_1 = Convolution1D(int(filter_size/2), kernel_size=4, activation=activations.relu, padding="valid")(img_1)
        img_1 = MaxPool1D(pool_size=2)(img_1)
        img_1 = Dropout(rate=0.1)(img_1)
        img_1 = Convolution1D(filter_size, kernel_size=2, activation=activations.relu, padding="valid")(img_1)
        img_1 = Convolution1D(filter_size, kernel_size=2, activation=activations.relu, padding="valid")(img_1)
        img_1 = MaxPool1D(pool_size=2)(img_1)
        img_1 = Dropout(rate=0.1)(img_1)
        
        dense = Dense(filter_size*2,activation=activations.relu, name="dense_1")(img_1)
        dense = Dropout(0.3)(dense)
        model = models.Model(inputs = inp, outputs = dense)
        return model

In [83]:

def get_model1(inputs_len = [187, 156 ,133 , 117], filter_sizes = [32, 16, 8]):
    nclass = 5
    
    inp_smallseq = Input(shape=(inputs_len[1], 1))
    inp_mediumseq = Input(shape=(inputs_len[2], 1))
    inp_largeseq = Input(shape=(inputs_len[3], 1))
    
    inp_smooth1 = Input(shape=(inputs_len[0], 1))   
    inp_smooth2 = Input(shape=(inputs_len[0], 1))   
    inp_smooth3 = Input(shape=(inputs_len[0], 1))   
    inp_org = Input(shape=(inputs_len[0], 1))
    
    base_net_org = get_base_model(inputs_len[0], filter_sizes[0])
    
    #smoothing
    base_net1 = get_base_model(inputs_len[0], filter_sizes[0])
    base_net2 = get_base_model(inputs_len[0], filter_sizes[0])
    base_net3 = get_base_model(inputs_len[0], filter_sizes[0])
    
    #sampling
    base_net_small = get_base_model(inputs_len[1], filter_sizes[0])
    base_net_medium = get_base_model(inputs_len[2], filter_sizes[0])
    base_net_large = get_base_model2(inputs_len[3], filter_sizes[0])
    
    embedding_org = base_net_org(inp_org)
    
    #smoothing
    embedding_smooth1 = base_net1(inp_smooth1)
    embedding_smooth2 = base_net2(inp_smooth2)
    embedding_smooth3 = base_net3(inp_smooth3)
    
    #sampling
    embedding_small = base_net_small(inp_smallseq)
    embedding_medium = base_net_medium(inp_mediumseq)
    embedding_large = base_net_large(inp_largeseq)
    
    # merge all the outputs
    merged1 = Concatenate(axis = 1)([embedding_org,embedding_smooth1, embedding_smooth2, embedding_smooth3, embedding_small, embedding_medium, embedding_large ]) 

    opt = optimizers.Adam(0.001) 
    
    merged1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(merged1)
    merged1 = GlobalMaxPool1D()(merged1)
    merged1 = Dropout(rate=0.2)(merged1)
    out1 = Dense(64, activation=activations.relu, name="dense_3_mitbih")(merged1)
    out1 = Dense(nclass, activation=activations.softmax, name="dense_4_mitbih")(out1)   

    model1 = models.Model(inputs=[inp_org,inp_smooth1, inp_smooth2, inp_smooth3, inp_smallseq, inp_mediumseq, inp_largeseq ], outputs=out1)
    model1.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model1.summary()

    return model1

In [84]:
model1 = get_model1(inputs_len = [187, 156 ,133 , 117], filter_sizes = [32, 16, 2])

file_path = "parallel_cnn1_mitbih.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_289 (InputLayer)          (None, 187, 1)       0                                            
__________________________________________________________________________________________________
input_286 (InputLayer)          (None, 187, 1)       0                                            
__________________________________________________________________________________________________
input_287 (InputLayer)          (None, 187, 1)       0                                            
__________________________________________________________________________________________________
input_288 (InputLayer)          (None, 187, 1)       0                                            
__________________________________________________________________________________________________
input_283 

In [86]:
X_train = [X, X_smooth_small, X_smooth_medium, X_smooth_large, X_sample_small, X_sample_medium, X_sample_large]
X_test_array = [X_test, X_test_smooth_small, X_test_smooth_medium, X_test_smooth_large, X_test_sample_small, X_test_sample_medium, X_test_sample_large]


#model.fit([X, X_smooth_small, X_smooth_medium, X_smooth_large], Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model1.fit(X_train, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model1.load_weights(file_path)

pred_test = model1.predict(X_test_array)
pred_test = np.argmax(pred_test, axis=-1)

Train on 78798 samples, validate on 8756 samples
Epoch 1/1000
 - 156s - loss: nan - acc: 0.8275 - val_loss: nan - val_acc: 0.8296

Epoch 00001: val_acc did not improve from 0.82960
Epoch 2/1000
 - 159s - loss: nan - acc: 0.8275 - val_loss: nan - val_acc: 0.8296

Epoch 00002: val_acc did not improve from 0.82960
Epoch 3/1000
 - 160s - loss: nan - acc: 0.8275 - val_loss: nan - val_acc: 0.8296

Epoch 00003: val_acc did not improve from 0.82960
Epoch 4/1000
 - 160s - loss: nan - acc: 0.8275 - val_loss: nan - val_acc: 0.8296

Epoch 00004: val_acc did not improve from 0.82960

Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Epoch 5/1000
 - 160s - loss: nan - acc: 0.8275 - val_loss: nan - val_acc: 0.8296

Epoch 00005: val_acc did not improve from 0.82960
Epoch 6/1000
 - 160s - loss: nan - acc: 0.8275 - val_loss: nan - val_acc: 0.8296

Epoch 00006: val_acc did not improve from 0.82960
Epoch 00006: early stopping


In [87]:
f1 = f1_score(Y_test, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.18113471632091976 
Test accuracy score : 0.8276082587246483 


In [88]:
def get_model2(inputs_len = [187, 156 ,133 , 117], filter_sizes = [32, 16, 8]):
    nclass = 5
    
    inp_smooth1 = Input(shape=(inputs_len[0], 1))   
    inp_smooth2 = Input(shape=(inputs_len[0], 1))   
    inp_smooth3 = Input(shape=(inputs_len[0], 1))   
    inp_org = Input(shape=(inputs_len[0], 1))
    
    base_net_org = get_base_model(inputs_len[0], filter_sizes[0])
    
    #smoothing
    base_net1 = get_base_model(inputs_len[0], filter_sizes[0])
    base_net2 = get_base_model(inputs_len[0], filter_sizes[0])
    base_net3 = get_base_model(inputs_len[0], filter_sizes[0])
    
    embedding_org = base_net_org(inp_org)
    
    #smoothing
    embedding_smooth1 = base_net1(inp_smooth1)
    embedding_smooth2 = base_net2(inp_smooth2)
    embedding_smooth3 = base_net3(inp_smooth3)
    
    # merge all the outputs
    merged2 = Concatenate(axis = 1)([embedding_org,embedding_smooth1, embedding_smooth2, embedding_smooth3]) 

    opt = optimizers.Adam(0.001) 
    
    merged2= Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(merged2)
    merged2 = GlobalMaxPool1D()(merged2)
    merged2 = Dropout(rate=0.2)(merged2)
    out2 = Dense(64, activation=activations.relu, name="dense_3_mitbih")(merged2)
    out2 = Dense(nclass, activation=activations.softmax, name="dense_4_mitbih")(out2)   

    model2 = models.Model(inputs=[inp_org,inp_smooth1, inp_smooth2, inp_smooth3], outputs=out2)
    model2.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model2.summary()
    
    return model2

In [89]:
model2 = get_model2(inputs_len = [187, 156 ,133 , 117], filter_sizes = [32, 16, 2])

file_path = "parallel_cnn2_mitbih.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_300 (InputLayer)          (None, 187, 1)       0                                            
__________________________________________________________________________________________________
input_297 (InputLayer)          (None, 187, 1)       0                                            
__________________________________________________________________________________________________
input_298 (InputLayer)          (None, 187, 1)       0                                            
__________________________________________________________________________________________________
input_299 (InputLayer)          (None, 187, 1)       0                                            
__________________________________________________________________________________________________
model_142 

In [90]:
X_train = [X, X_smooth_small, X_smooth_medium, X_smooth_large]

#model.fit([X, X_smooth_small, X_smooth_medium, X_smooth_large], Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model2.fit(X_train, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model2.load_weights(file_path)

Train on 78798 samples, validate on 8756 samples
Epoch 1/1000
 - 125s - loss: nan - acc: 0.8272 - val_loss: nan - val_acc: 0.8296

Epoch 00001: val_acc improved from -inf to 0.82960, saving model to parallel_cnn2_mitbih.h5
Epoch 2/1000
 - 121s - loss: nan - acc: 0.8275 - val_loss: nan - val_acc: 0.8296

Epoch 00002: val_acc did not improve from 0.82960
Epoch 3/1000
 - 121s - loss: nan - acc: 0.8275 - val_loss: nan - val_acc: 0.8296

Epoch 00003: val_acc did not improve from 0.82960
Epoch 4/1000
 - 120s - loss: nan - acc: 0.8275 - val_loss: nan - val_acc: 0.8296

Epoch 00004: val_acc did not improve from 0.82960

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 5/1000
 - 128s - loss: nan - acc: 0.8275 - val_loss: nan - val_acc: 0.8296

Epoch 00005: val_acc did not improve from 0.82960
Epoch 6/1000
 - 128s - loss: nan - acc: 0.8275 - val_loss: nan - val_acc: 0.8296

Epoch 00006: val_acc did not improve from 0.82960
Epoch 00006: early stopping


ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 4 array(s), but instead got the following list of 1 arrays: [array([[[1.        ],
        [0.75826448],
        [0.11157025],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.90842491],
        [0.7838828 ],
        ...

In [92]:
X_test_array = [X_test, X_test_smooth_small, X_test_smooth_medium, X_test_smooth_large]

pred_test = model2.predict(X_test_array)
pred_test = np.argmax(pred_test, axis=-1)

In [93]:
f1 = f1_score(Y_test, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.18113471632091976 
Test accuracy score : 0.8276082587246483 


In [94]:
def get_model3(inputs_len = [187, 156 ,133 , 117], filter_sizes = [ 32, 16, 8]):
    nclass = 5
    
    inp_smallseq = Input(shape=(inputs_len[1], 1))
    inp_mediumseq = Input(shape=(inputs_len[2], 1))
    inp_largeseq = Input(shape=(inputs_len[3], 1))  
    inp_org = Input(shape=(inputs_len[0], 1))
    
    base_net_org = get_base_model(inputs_len[0], filter_sizes[0])
    
    #sampling
    base_net_small = get_base_model(inputs_len[1], filter_sizes[0])
    base_net_medium = get_base_model(inputs_len[2], filter_sizes[0])
    base_net_large = get_base_model2(inputs_len[3], filter_sizes[0])
    
    embedding_org = base_net_org(inp_org)
    
    #sampling
    embedding_small = base_net_small(inp_smallseq)
    embedding_medium = base_net_medium(inp_mediumseq)
    embedding_large = base_net_large(inp_largeseq)
    
    # merge all the outputs
    merged3 = Concatenate(axis = 1)([embedding_org,embedding_small, embedding_medium, embedding_large ]) 
    
    opt = optimizers.Adam(0.001) 
       
    merged3= Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(merged3)
    merged3 = GlobalMaxPool1D()(merged3)
    merged3 = Dropout(rate=0.2)(merged3)
    out3 = Dense(64, activation=activations.relu, name="dense_3_mitbih")(merged3)
    out3 = Dense(nclass, activation=activations.softmax, name="dense_4_mitbih")(out3)   

    model3 = models.Model(inputs=[inp_org,inp_smallseq, inp_mediumseq, inp_largeseq], outputs=out3)
    model3.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model3.summary()
   
    return model3

In [95]:
model3 = get_model3(inputs_len = [187, 156 ,133 , 117], filter_sizes = [ 32, 16, 2])

file_path = "parallel_cnn3_mitbih.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_308 (InputLayer)          (None, 187, 1)       0                                            
__________________________________________________________________________________________________
input_305 (InputLayer)          (None, 156, 1)       0                                            
__________________________________________________________________________________________________
input_306 (InputLayer)          (None, 133, 1)       0                                            
__________________________________________________________________________________________________
input_307 (InputLayer)          (None, 117, 1)       0                                            
__________________________________________________________________________________________________
model_147 

In [98]:
X_train = [X, X_sample_small, X_sample_medium, X_sample_large]
X_test_array = [X_test, X_test_sample_small, X_test_sample_medium, X_test_sample_large]

#model.fit([X, X_smooth_small, X_smooth_medium, X_smooth_large], Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model3.fit(X_train, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model3.load_weights(file_path)

pred_test = model3.predict(X_test_array)
pred_test = np.argmax(pred_test, axis=-1)

Train on 78798 samples, validate on 8756 samples
Epoch 1/1000
 - 126s - loss: 0.1770 - acc: 0.9519 - val_loss: 0.1405 - val_acc: 0.9613

Epoch 00001: val_acc improved from 0.95066 to 0.96128, saving model to parallel_cnn3_mitbih.h5
Epoch 2/1000
 - 125s - loss: 0.1540 - acc: 0.9582 - val_loss: 0.1213 - val_acc: 0.9672

Epoch 00002: val_acc improved from 0.96128 to 0.96722, saving model to parallel_cnn3_mitbih.h5
Epoch 3/1000
 - 126s - loss: 0.1378 - acc: 0.9625 - val_loss: 0.1119 - val_acc: 0.9711

Epoch 00003: val_acc improved from 0.96722 to 0.97111, saving model to parallel_cnn3_mitbih.h5
Epoch 4/1000
 - 128s - loss: 0.1264 - acc: 0.9661 - val_loss: 0.1065 - val_acc: 0.9717

Epoch 00004: val_acc improved from 0.97111 to 0.97168, saving model to parallel_cnn3_mitbih.h5
Epoch 5/1000
 - 126s - loss: 0.1187 - acc: 0.9680 - val_loss: 0.1034 - val_acc: 0.9730

Epoch 00005: val_acc improved from 0.97168 to 0.97305, saving model to parallel_cnn3_mitbih.h5
Epoch 6/1000
 - 126s - loss: 0.1122 

In [99]:
f1 = f1_score(Y_test, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.9038775701760254 
Test accuracy score : 0.9835556367622876 


In [100]:
def get_model4(inputs_len = [187, 156 ,133 , 117], filter_sizes = [ 32, 16, 8]):
    nclass = 5
    
    inp_smallseq = Input(shape=(inputs_len[1], 1))
    
    inp_smooth1 = Input(shape=(inputs_len[0], 1))   
    inp_org = Input(shape=(inputs_len[0], 1))
    
    base_net_org = get_base_model(inputs_len[0], filter_sizes[0])
    
    #smoothing
    base_net1 = get_base_model(inputs_len[0], filter_sizes[0])
    
    #sampling
    base_net_small = get_base_model(inputs_len[1], filter_sizes[0])
    
    embedding_org = base_net_org(inp_org)
    
    #smoothing
    embedding_smooth1 = base_net1(inp_smooth1)
    
    #sampling
    embedding_small = base_net_small(inp_smallseq)
    
    # merge all the outputs
    merged4 = Concatenate(axis = 1)([embedding_org,embedding_smooth1, embedding_small ]) 

    opt = optimizers.Adam(0.001) 

    merged4= Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(merged4)
    merged4 = GlobalMaxPool1D()(merged4)
    merged4 = Dropout(rate=0.2)(merged4)
    out4 = Dense(64, activation=activations.relu, name="dense_3_mitbih")(merged4)
    out4 = Dense(nclass, activation=activations.softmax, name="dense_4_mitbih")(out4)   

    model4 = models.Model(inputs=[inp_org,inp_smooth1,inp_smallseq], outputs=out4)
    model4.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model4.summary()  
    
    return model4

In [101]:
model4 = get_model4(inputs_len = [187, 156 ,133 , 117], filter_sizes = [ 32, 16, 2])

file_path = "parallel_cnn4_mitbih.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_315 (InputLayer)          (None, 187, 1)       0                                            
__________________________________________________________________________________________________
input_314 (InputLayer)          (None, 187, 1)       0                                            
__________________________________________________________________________________________________
input_313 (InputLayer)          (None, 156, 1)       0                                            
__________________________________________________________________________________________________
model_152 (Model)               (None, 20, 64)       12064       input_315[0][0]                  
__________________________________________________________________________________________________
model_153 

In [102]:
X_train = [X, X_smooth_small, X_sample_small]
X_test_array = [X_test, X_test_smooth_small, X_test_sample_small]

#model.fit([X, X_smooth_small, X_smooth_medium, X_smooth_large], Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model4.fit(X_train, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model4.load_weights(file_path)

pred_test = model4.predict(X_test_array)
pred_test = np.argmax(pred_test, axis=-1)

Train on 78798 samples, validate on 8756 samples
Epoch 1/1000
 - 126s - loss: nan - acc: 0.8340 - val_loss: nan - val_acc: 0.8296

Epoch 00001: val_acc improved from -inf to 0.82960, saving model to parallel_cnn4_mitbih.h5
Epoch 2/1000
 - 122s - loss: nan - acc: 0.8275 - val_loss: nan - val_acc: 0.8296

Epoch 00002: val_acc did not improve from 0.82960
Epoch 3/1000
 - 123s - loss: nan - acc: 0.8275 - val_loss: nan - val_acc: 0.8296

Epoch 00003: val_acc did not improve from 0.82960
Epoch 4/1000
 - 122s - loss: nan - acc: 0.8275 - val_loss: nan - val_acc: 0.8296

Epoch 00004: val_acc did not improve from 0.82960

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 5/1000
 - 133s - loss: nan - acc: 0.8275 - val_loss: nan - val_acc: 0.8296

Epoch 00005: val_acc did not improve from 0.82960
Epoch 6/1000
 - 133s - loss: nan - acc: 0.8275 - val_loss: nan - val_acc: 0.8296

Epoch 00006: val_acc did not improve from 0.82960
Epoch 00006: early stopping


ValueError: Error when checking input: expected input_245 to have shape (47, 1) but got array with shape (156, 1)

In [103]:
pred_test = model4.predict(X_test_array)
pred_test = np.argmax(pred_test, axis=-1)

In [104]:
f1 = f1_score(Y_test, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.18113471632091976 
Test accuracy score : 0.8276082587246483 
